![SolidQ](https://antoniosql.github.io/images/SolidQ_Verne.png) 
#Interpretabilidad de Modelos de Machine Learning con Interpret 

## ¿Por qué necesitamos Interpretar Modelos?

Hay quien dice que si los resultados son correctos, ¿qué importa como se llegue a ellos? Olvidando que como humanos que somos la lógica y el razonamiento va siempre de la mano de nuestras decisiones. Además, si tratamos nuestros modelos como cajas negras, será complicado detectar posibles problemas, tales como los sesgos o el sobreentrenamiento. 

Como científicos de datos, necesitamos entender el por qué un determinado modelo arroja un resultado. Además, necesitamos poder explicar al usuario final por qué toma el modelo sus decisiones. A mayores de todo esto, debemos de hacer un ML responsable que evite sesgos de cualquier tipo. Por todo ello, se hace indispensable la interpretabilidad. Podríamos resumir las necesidades en:
- Legales. GDPR incorpora determinados requisitos
- Morales. Para evitar sesgos y asegurarnos que el modelo toma decisiones justas.
- De Negocio. Saber interpretar por qué el modelo toma sus decisiones y que características del proceso son las que tienen mayor relevancia. 

## ¿Por qué Interpret?
Es un paquete que encapsula muchas de las técnicas actuales de interpretabilidad de modelos de ML. Interpret clasifa los modelos en:
- Glassbox. Son modelos que parten de algoritmos diseñados para ser interpretables, como los Árboles de Decisión o los modelos Lineales, que habitualmente proporcionan predicciones exactas
- Blackbox. Son modelos cuya interpretabilidad no es trivial, y es necesario utilizar determinadas técnicas para extraer esa información. 

En este ejemplo nos centraremos en un modelo blackbox, que será un Clasificador de RandonForest. Utilizaremos uno de los conjuntos de datos más utilizados en ML para demostrar técnicas de clasificación, que son los datos de Censo de adultos del año 1994, para intentar predecir si los ingresos de una persona son superiores a 50k USD o no. 

## El código

### Instalación y carga de paquetes

In [ ]:
#Instalando el paquete interpret

!pip install interpret

In [15]:
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

### Carga del Conjunto de datos y Transformaciones básicas

In [16]:
#Cargar Datos
# Descripción del conjunto de datos: https://archive.ics.uci.edu/ml/datasets/Adult

df = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",header=None)
df.columns = [
    "Age", "WorkClass", "fnlwgt", "Education", "EducationNum",
    "MaritalStatus", "Occupation", "Relationship", "Race", "Gender",
    "CapitalGain", "CapitalLoss", "HoursPerWeek", "NativeCountry", "Income"]
train_cols = df.columns[0:-1]
label = df.columns[-1]
X = df[train_cols]
y = df[label].apply(lambda x: 0 if x == " <=50K" else 1) #convertimos en 0 o 1 si es >50k

In [17]:
# Transformamos las variables categóricas
X_enc = pd.get_dummies(X, prefix_sep='.')
feature_names = list(X_enc.columns)

seed = 1  
X_train, X_test, y_train, y_test = train_test_split(X_enc, y, test_size=0.20, random_state=seed)


### Creamos el pipeline y entrenamos el Clasificador

In [18]:

#Entrenamos el clasificador
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

#Un sistema Blackbox puede incluir preprocesado, no solo el clasificador

pca = PCA()
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1)

blackbox_model = Pipeline([('pca', pca), ('rf', rf)])
blackbox_model.fit(X_train, y_train)



Pipeline(memory=None,
         steps=[('pca',
                 PCA(copy=True, iterated_power='auto', n_components=None,
                     random_state=None, svd_solver='auto', tol=0.0,
                     whiten=False)),
                ('rf',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=-1,
                                        oob_score=False, random_state=None,
                    

### ¿Cómo de bueno es el modelo? Rendimiento

In [27]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
pred = blackbox_model.predict(X_test)
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred))

[[4673  353]
 [ 615  872]]
              precision    recall  f1-score   support

           0       0.88      0.93      0.91      5026
           1       0.71      0.59      0.64      1487

    accuracy                           0.85      6513
   macro avg       0.80      0.76      0.77      6513
weighted avg       0.84      0.85      0.85      6513



In [28]:
from interpret import show
from interpret.perf import ROC

blackbox_perf = ROC(blackbox_model.predict_proba).explain_perf(X_test, y_test, name='Modelo de Caja Negra')
show(blackbox_perf)

### Interpretación Local con LIME
La idea es bastante intuitiva. Primero, olvídate de los datos de entrenamiento e imagina que solo tienes el modelo de caja negra donde puedes ingresar puntos de datos y obtener las predicciones del modelo. Puede sondear la caja con la frecuencia que desee. Su objetivo es entender por qué el modelo de aprendizaje automático hizo una cierta predicción. LIME prueba lo que sucede con las predicciones cuando se proporcionan variaciones de los datos en el modelo de aprendizaje automático. LIME genera un nuevo conjunto de datos que consta de muestras persilenciadas y las predicciones correspondientes del modelo de caja negra. En este nuevo DataSet LIME, a continuación, entrena un modelo interpretables, que se pondera por la proximidad de las instancias muestreadas a la instancia de interés. El modelo interpretables puede ser cualquier cosa del capítulo de modelos interpretables, por ejemplo Lasso o un árbol de decisión. El modelo aprendido debe ser una buena aproximación de las predicciones del modelo de aprendizaje automático localmente, pero no tiene que ser una buena aproximación global. Este tipo de precisión también se denomina fidelidad local.


In [20]:
from interpret.blackbox import LimeTabular
from interpret import show

#Los "explicadores" de cajas negras necesitan una función de predicción, y opcionalmente un conjunto de datos
lime = LimeTabular(predict_fn=blackbox_model.predict_proba, data=X_train)

#Seleccionamos las instancias a explicar, y podmeos pasar opcionalmente las etiquetas si las tenemos
lime_local = lime.explain_local(X_test[:5], y_test[:5], name='LIME')

show(lime_local)

In [21]:
from interpret.blackbox import ShapKernel
import numpy as np

background_val = np.median(X_train, axis=0).reshape(1, -1)
shap = ShapKernel(predict_fn=blackbox_model.predict_proba, data=background_val, feature_names=feature_names)
shap_local = shap.explain_local(X_test[:5], y_test[:5], name='SHAP')
show(shap_local)

### Intepretabilidad Global con Sensibilidad de Morris

El método comienza muestreando un conjunto de valores iniciales dentro de los rangos definidos de valores posibles para todas las variables de entrada y calculando el resultado del modelo. El segundo paso cambia los valores de una variable (todas las demás entradas restantes en sus valores iniciales) y calcula el cambio resultante en el resultado del modelo en comparación con la primera ejecución. A continuación, se cambian los valores de otra variable (la variable anterior se mantiene en su valor modificado y todas las demás se mantienen en sus valores iniciales) y se calcula el cambio resultante en el resultado del modelo en comparación con la segunda ejecución. Esto continúa hasta que se cambian todas las variables de entrada. Este procedimiento se repite r veces (donde se toman generalmente entre 5 y 15), cada vez con un conjunto diferente de valores de inicio, lo que conduce a un número de r(k + 1) corridas, donde k es el número de variables de entrada.

In [22]:

from interpret.blackbox import MorrisSensitivity

sensitivity = MorrisSensitivity(predict_fn=blackbox_model.predict_proba, data=X_train)
sensitivity_global = sensitivity.explain_global(name="Sensibilidad de Morris")

show(sensitivity_global)

### Intepretabilidad Global con PDP
La gráfica de dependencia parcial (gráfica de PDP o PD corta) muestra el efecto marginal que una o dos características tienen en el resultado pronosticado de un modelo de aprendizaje automático (j. h. Friedman 200127). Una gráfica de dependencia parcial puede mostrar si la relación entre el destino y una entidad es lineal, monótona o más compleja. Por ejemplo, cuando se aplica a un modelo de regresión lineal, los trazados de dependencia parcial siempre muestran una relación lineal.


In [23]:
from interpret.blackbox import PartialDependence

pdp = PartialDependence(predict_fn=blackbox_model.predict_proba, data=X_train)
pdp_global = pdp.explain_global(name='Dependencias Parciales')

show(pdp_global)